# Scrapping

In [1]:
!pip install google_play_scraper
from google_play_scraper import app, Sort, reviews_all #import librarry untuk scrapping data dari play store
import csv
import pandas as pd

In [2]:
scrap_data = reviews_all(
    'com.kai.kaiticketing',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count = 10000
)

In [3]:
scrap_data = scrap_data[:15000]

In [4]:
with open('review_kai_data.csv', mode='w', newline='', encoding='utf-8')as file:
    writer = csv.writer(file)
    writer.writerow(['review'])
    for scrap in scrap_data:
        writer.writerow([scrap['content']])

In [5]:
app_review = pd.DataFrame(scrap_data)

In [6]:
app_review.shape

(15000, 11)

In [7]:
app_review.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a105e9e0-a60b-4078-b9be-15bce3ea609a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi KAI Acces sebelumnya lebih simple dan...,1,2234,6.4.1,2024-04-24 23:29:53,None,NaT,6.4.1
1,6c39952f-1ff5-44d8-bcec-9f78383a89a2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Entah kenapa appnya yg update terbaru sekarang...,1,18,6.9.11,2025-01-12 14:21:11,None,NaT,6.9.11
2,c0ec6dfd-d0ea-437f-b4a2-fff297e9f92f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Alasan Rating: Tidak kompatibel dengan ponsel ...,3,0,6.9.13,2025-01-13 06:42:24,None,NaT,6.9.13
3,0b2379be-0534-454b-b53f-efc37cfb926d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Setelah update terakhir ko malah susah banget ...,3,8,6.9.11,2025-01-06 04:36:23,"Hi kak, mohon maaf atas kendalanya, aplikasi a...",2023-09-13 01:01:04,6.9.11
4,41706ef4-3a74-45cf-a2a4-d57af749ece7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Menurut aku sih secara UI udah rapih dan mudah...,1,157,6.9.11,2024-12-27 12:19:44,None,NaT,6.9.11


In [8]:
app_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              15000 non-null  object        
 1   userName              15000 non-null  object        
 2   userImage             15000 non-null  object        
 3   content               15000 non-null  object        
 4   score                 15000 non-null  int64         
 5   thumbsUpCount         15000 non-null  int64         
 6   reviewCreatedVersion  12584 non-null  object        
 7   at                    15000 non-null  datetime64[ns]
 8   replyContent          2379 non-null   object        
 9   repliedAt             2379 non-null   datetime64[ns]
 10  appVersion            12584 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 1.3+ MB


# Cleaning

In [9]:
app_review.duplicated().sum() #Check Missing Value

0

In [10]:
app_review.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,2416
at,0
replyContent,12621
repliedAt,12621


**Insight**
- Karena saya hanya butuh kolom content untuk menganalisa sentiment dari aplikasi maka saya memutuskan untuk drop column replycontent dan repliedat, karena memiliki banyak missing value yang memungkinkan saya akan kehilangan banyak data bila saya tidak drop column itu pada tahap pembersihan MV

In [11]:
drop_col = ['replyContent', 'repliedAt']
app_review = app_review.drop(drop_col, axis=1)
app_review.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,2416
at,0
appVersion,2416


In [12]:
data_clean = app_review.dropna()
data_clean.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,0
at,0
appVersion,0


In [13]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12584 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              12584 non-null  object        
 1   userName              12584 non-null  object        
 2   userImage             12584 non-null  object        
 3   content               12584 non-null  object        
 4   score                 12584 non-null  int64         
 5   thumbsUpCount         12584 non-null  int64         
 6   reviewCreatedVersion  12584 non-null  object        
 7   at                    12584 non-null  datetime64[ns]
 8   appVersion            12584 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 983.1+ KB


**Insight**
- Sampai saat ini data sudah bersih dari MV dan DP, sehingga sudah bisa saya save ke format csv untuk analisa lebih lanjut

# Text Preprocessing

In [14]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12584 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              12584 non-null  object        
 1   userName              12584 non-null  object        
 2   userImage             12584 non-null  object        
 3   content               12584 non-null  object        
 4   score                 12584 non-null  int64         
 5   thumbsUpCount         12584 non-null  int64         
 6   reviewCreatedVersion  12584 non-null  object        
 7   at                    12584 non-null  datetime64[ns]
 8   appVersion            12584 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 983.1+ KB


In [15]:
data_content = data_clean[['content', 'appVersion']]
data_content

,content,appVersion
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13
3,Setelah update terakhir ko malah susah banget ...,6.9.11
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11
...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2


## Case Folding

In [16]:
data_content['text_lower'] = data_content['content'].str.lower()
data_content

<ipython-input-16-44973df7654e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content['text_lower'] = data_content['content'].str.lower()


,content,appVersion,text_lower
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...
...,...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2,"kok akun ga bisa masuk masuk kayak ke hack , p..."
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1,"aplikasinya udah bagus, memudahkan saya dalam ..."
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0,kecewa sih udah berhasil melakukan pembayaran ...
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2,"baru aja download, abis ngisi"" data dan berhas..."


## Remove Slangword


In [17]:
slangwords = {"@": "di", "abis": "sesudah", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "joss":"baik", "kok":"kenapa", "udah":"sudah", "min":"admin", "yg":"yang", "ui":"tampilan", "klu":"kalau"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

## Remove Special Char

In [18]:
import re
import string
!pip install emoji
import emoji

In [19]:
def rmv_special_char(text):
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r"(.)\1\1+", r"\1\1", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    text = emoji.replace_emoji(text, replace='')
    return text

In [20]:
data_content['removed_special_char'] = data_content['text_lower'].apply(rmv_special_char)
data_content

<ipython-input-20-300163b2556e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content['removed_special_char'] = data_content['text_lower'].apply(rmv_special_char)


,content,appVersion,text_lower,removed_special_char
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yg update terbaru sekarang...
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...,alasan rating tidak kompatibel dengan ponsel s...
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara ui udah rapih dan mudah...
...,...,...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2,"kok akun ga bisa masuk masuk kayak ke hack , p...",kok akun ga bisa masuk masuk kayak ke hack pe...
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1,"aplikasinya udah bagus, memudahkan saya dalam ...",aplikasinya udah bagus memudahkan saya dalam m...
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih udah berhasil melakukan pembayaran ...
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2,"baru aja download, abis ngisi"" data dan berhas...",baru aja download abis ngisi data dan berhasil...


In [21]:
data_content['remove_slang'] = data_content['removed_special_char'].apply(fix_slangwords)
data_content

<ipython-input-21-895c27409e86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content['remove_slang'] = data_content['removed_special_char'].apply(fix_slangwords)


,content,appVersion,text_lower,removed_special_char,remove_slang
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yang update terbaru sekara...
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...,alasan rating tidak kompatibel dengan ponsel s...,alasan rating tidak kompatibel dengan ponsel s...
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara tampilan sudah rapih da...
...,...,...,...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2,"kok akun ga bisa masuk masuk kayak ke hack , p...",kok akun ga bisa masuk masuk kayak ke hack pe...,kenapa akun ga bisa masuk masuk kayak ke hack ...
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1,"aplikasinya udah bagus, memudahkan saya dalam ...",aplikasinya udah bagus memudahkan saya dalam m...,aplikasinya sudah bagus memudahkan saya dalam ...
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih sudah berhasil melakukan pembayaran...
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2,"baru aja download, abis ngisi"" data dan berhas...",baru aja download abis ngisi data dan berhasil...,baru aja download sesudah ngisi data dan berha...


In [22]:
data_content.isnull().sum()

,0
content,0
appVersion,0
text_lower,0
removed_special_char,0
remove_slang,0


In [23]:
data_content.head()

,content,appVersion,text_lower,removed_special_char,remove_slang
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yang update terbaru sekara...
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...,alasan rating tidak kompatibel dengan ponsel s...,alasan rating tidak kompatibel dengan ponsel s...
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara tampilan sudah rapih da...


## Tokenization

In [24]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
def tokenized_word(text):
  text = word_tokenize(text)
  return text

In [27]:
data_content['tokenized_word'] = data_content['remove_slang'].apply(tokenized_word)
data_content

<ipython-input-27-da98c2de19cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content['tokenized_word'] = data_content['remove_slang'].apply(tokenized_word)


,content,appVersion,text_lower,removed_special_char,remove_slang,tokenized_word
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,"[aplikasi, kai, acces, sebelumnya, lebih, simp..."
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yang update terbaru sekara...,"[entah, kenapa, appnya, yang, update, terbaru,..."
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...,alasan rating tidak kompatibel dengan ponsel s...,alasan rating tidak kompatibel dengan ponsel s...,"[alasan, rating, tidak, kompatibel, dengan, po..."
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,"[setelah, update, terakhir, ko, malah, susah, ..."
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara tampilan sudah rapih da...,"[menurut, aku, sih, secara, tampilan, sudah, r..."
...,...,...,...,...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2,"kok akun ga bisa masuk masuk kayak ke hack , p...",kok akun ga bisa masuk masuk kayak ke hack pe...,kenapa akun ga bisa masuk masuk kayak ke hack ...,"[kenapa, akun, ga, bisa, masuk, masuk, kayak, ..."
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1,"aplikasinya udah bagus, memudahkan saya dalam ...",aplikasinya udah bagus memudahkan saya dalam m...,aplikasinya sudah bagus memudahkan saya dalam ...,"[aplikasinya, sudah, bagus, memudahkan, saya, ..."
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih sudah berhasil melakukan pembayaran...,"[kecewa, sih, sudah, berhasil, melakukan, pemb..."
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2,"baru aja download, abis ngisi"" data dan berhas...",baru aja download abis ngisi data dan berhasil...,baru aja download sesudah ngisi data dan berha...,"[baru, aja, download, sesudah, ngisi, data, da..."


## Stopword

In [28]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
def stopword_func(text):
  list_stopword = set(stopwords.words('indonesian'))
  list_stopword.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
  filtered = []
  for txt in text:
    if txt not in list_stopword:
      filtered.append(txt)
  text=filtered
  return text

In [30]:
data_content['stopwords_removed'] = data_content['tokenized_word'].apply(stopword_func)
data_content

<ipython-input-30-011efc511dfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content['stopwords_removed'] = data_content['tokenized_word'].apply(stopword_func)


,content,appVersion,text_lower,removed_special_char,remove_slang,tokenized_word,stopwords_removed
0,Aplikasi KAI Acces sebelumnya lebih simple dan...,6.4.1,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,aplikasi kai acces sebelumnya lebih simple dan...,"[aplikasi, kai, acces, sebelumnya, lebih, simp...","[aplikasi, kai, acces, simple, langsung, lanca..."
1,Entah kenapa appnya yg update terbaru sekarang...,6.9.11,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yg update terbaru sekarang...,entah kenapa appnya yang update terbaru sekara...,"[entah, kenapa, appnya, yang, update, terbaru,...","[appnya, update, terbaru, error, sistemnya, ka..."
2,Alasan Rating: Tidak kompatibel dengan ponsel ...,6.9.13,alasan rating: tidak kompatibel dengan ponsel ...,alasan rating tidak kompatibel dengan ponsel s...,alasan rating tidak kompatibel dengan ponsel s...,"[alasan, rating, tidak, kompatibel, dengan, po...","[alasan, rating, kompatibel, ponsel, sayasaya,..."
3,Setelah update terakhir ko malah susah banget ...,6.9.11,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,setelah update terakhir ko malah susah banget ...,"[setelah, update, terakhir, ko, malah, susah, ...","[update, ko, susah, banget, login, siih, bbrp,..."
4,Menurut aku sih secara UI udah rapih dan mudah...,6.9.11,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara ui udah rapih dan mudah...,menurut aku sih secara tampilan sudah rapih da...,"[menurut, aku, sih, secara, tampilan, sudah, r...","[tampilan, rapih, mudah, pemesanan, tiket, uta..."
...,...,...,...,...,...,...,...
14995,"kok akun ga bisa masuk masuk kayak ke hack , p...",6.3.2,"kok akun ga bisa masuk masuk kayak ke hack , p...",kok akun ga bisa masuk masuk kayak ke hack pe...,kenapa akun ga bisa masuk masuk kayak ke hack ...,"[kenapa, akun, ga, bisa, masuk, masuk, kayak, ...","[akun, masuk, masuk, kayak, hack, perbarui, sa..."
14996,"Aplikasinya udah bagus, memudahkan saya dalam ...",4.0.1,"aplikasinya udah bagus, memudahkan saya dalam ...",aplikasinya udah bagus memudahkan saya dalam m...,aplikasinya sudah bagus memudahkan saya dalam ...,"[aplikasinya, sudah, bagus, memudahkan, saya, ...","[aplikasinya, bagus, memudahkan, memesan, tike..."
14997,Kecewa sih udah berhasil melakukan pembayaran ...,4.9.0,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih udah berhasil melakukan pembayaran ...,kecewa sih sudah berhasil melakukan pembayaran...,"[kecewa, sih, sudah, berhasil, melakukan, pemb...","[kecewa, berhasil, pembayaran, shopeepay, tike..."
14998,"baru aja download, abis ngisi"" data dan berhas...",6.3.2,"baru aja download, abis ngisi"" data dan berhas...",baru aja download abis ngisi data dan berhasil...,baru aja download sesudah ngisi data dan berha...,"[baru, aja, download, sesudah, ngisi, data, da...","[aja, download, ngisi, data, berhasil, eh, dap..."


## Steming using Sastrawi

In [31]:
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# Inisialisasi Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemming untuk setiap list token di kolom
data_content["stemmed_word"] = data_content["stopwords_removed"].apply(lambda x: [stemmer.stem(token) for token in x])
data_content

# Join Sentence

In [ ]:
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

data_content['Final_content'] = data_content['stopwords_removed'].apply(toSentence)
data_content

# Labelling

In [ ]:
import csv
import requests
from io import StringIO

lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Gagal Fetch Negative")

In [ ]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    else:
        polarity = 'neutral'

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [ ]:
results = data_content['stopwords_removed'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
data_content['polarity_score'] = results[0]
data_content['polarity'] = results[1]
print(data_content['polarity'].value_counts())

In [ ]:
data_content

In [ ]:
data_content.to_csv('data_siap_process.csv', index=False)